In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=2, dirichlet="bottom|right")
gfu = GridFunction(fes)
a = BilinearForm(fes, symmetric=True)
a += grad(fes.TrialFunction())*grad(fes.TestFunction())*dx
a.Assemble()
f = LinearForm(fes)
f += x*fes.TestFunction()*dx
f.Assemble()
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec

In [2]:
vtk = VTKOutput(mesh,coefs=[gfu],names=["sol"],filename="vtk_example1",subdivision=2)
vtk.Do()

'vtk_example1'

In [9]:
from ngsolve.webgui import Draw
#Draw(gfu)

In [4]:
mesh.edges

In [4]:
class Element:
    def __init__(self, node_indexes):
        self.node_indexes = node_indexes
        self.edges = []  # List to store edges in the element

    def repr(self):
        return f"Element({self.node_indexes})"

class Edge:
    def __init__(self, node1_index, node2_index):
        # Ensure edge is stored in sorted order to avoid duplicates like (0, 1) and (1, 0)
        self.node1_index = min(node1_index, node2_index)
        self.node2_index = max(node1_index, node2_index)
        self.neighbors = []  # List of neighboring elements that share this edge

    def add_neighbor(self, element):
        self.neighbors.append(element)

    def repr(self):
        return f"Edge({self.node1_index}, {self.node2_index})"

    def eq(self, other):
        return (self.node1_index == other.node1_index and self.node2_index == other.node2_index)

    def hash(self):
        return hash((self.node1_index, self.node2_index))

class Edges:
    def __init__(self):
        self.edges = set()  # Using a set to automatically handle uniqueness

    def add_edge(self, edge, element):
        # Check if the edge already exists; if it does, just add the element as a neighbor
        if edge not in self.edges:
            self.edges.add(edge)
        edge.add_neighbor(element)

    def generate_edges_from_elements(self, elements):
        for element in elements:
            nodes = element.node_indexes
            for i in range(len(nodes)):
                node1 = nodes[i]
                node2 = nodes[(i + 1) % len(nodes)]  # Wrap around to form a closed loop
                edge = Edge(node1, node2)
                self.add_edge(edge, element)

    def repr(self):
        return f"Edges({list(self.edges)})"

# Example usage
elements = [
    Element([0, 1, 2]),  # A triangle with nodes 0, 1, 2
    Element([2, 3, 4]),  # A triangle with nodes 2, 3, 4
    Element([1, 2, 5])   # Another triangle with nodes 1, 2, 5
]

edges = Edges()
edges.generate_edges_from_elements(elements)

# Display the edges and their neighbors
for edge in edges.edges:
    print(f"{edge} -> Neighbors: {edge.neighbors}")

<__main__.Edge object at 0x0000022FA9F28C40> -> Neighbors: [<__main__.Element object at 0x0000022FA9BB5E50>]
<__main__.Edge object at 0x0000022FA9F28E80> -> Neighbors: [<__main__.Element object at 0x0000022FA8A08F10>]
<__main__.Edge object at 0x0000022FA9F28A90> -> Neighbors: [<__main__.Element object at 0x0000022FA9BA0610>]
<__main__.Edge object at 0x0000022FA9F28CA0> -> Neighbors: [<__main__.Element object at 0x0000022FA9BB5E50>]
<__main__.Edge object at 0x0000022FA9F28AC0> -> Neighbors: [<__main__.Element object at 0x0000022FA9BA0610>]
<__main__.Edge object at 0x0000022FA9F28D00> -> Neighbors: [<__main__.Element object at 0x0000022FA8A08F10>]
<__main__.Edge object at 0x0000022FA9F28D60> -> Neighbors: [<__main__.Element object at 0x0000022FA8A08F10>]
<__main__.Edge object at 0x0000022FA9F28B80> -> Neighbors: [<__main__.Element object at 0x0000022FA9BA0610>]
<__main__.Edge object at 0x0000022FA9F28BE0> -> Neighbors: [<__main__.Element object at 0x0000022FA9BB5E50>]
